# Converting File

In [ ]:
import h5py
import numpy as np
import pandas as pd

# === Load data from HDF5-based .mat file
with h5py.File("data_s1_64.mat") as f:
    raw = f['datas'][:]  # shape: (12, 60, 5140, 64, 2)

# Transpose: (mod, ch, time, freq, trial)
data = np.transpose(raw, (4, 3, 2, 1, 0))  # shape: (2, 64, 5140, 60, 12)

# === Settings
channel_indices = [9, 25, 27, 29, 47, 52, 58, 61]           # Your 8 channels
frequency_map = {7: 6, 8: 7, 10: 9, 13: 12}  # Hz → index
mod_index = 1  # High contrast

# === Extract and save each frequency
subset = data[mod_index]  # shape: (64, 5140, 60, 12)

for hz, f_idx in frequency_map.items():
    # Get data: (8 ch, 5140 time, 12 trials)
    block = subset[channel_indices, :, f_idx, :]  # shape: (8, 5140, 12)

    # Prepare one column per channel, stacking 12 trials vertically
    columns = []
    for ch in range(8):
        trials = block[ch]  # shape: (5140, 12)
        full_column = trials.T.reshape(-1)  # Stack trials: (12 × 5140,)
        columns.append(full_column)

    # Combine all columns
    df = pd.DataFrame(np.column_stack(columns),
                      columns=['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'PO8', 'Oz'])

    # Save to CSV
    df.to_csv(f"S1_freq_{hz}Hz.csv", index=False)

print("✅ Done: Saved 4 CSVs, one per frequency")


# Plotting a File

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import iirnotch, filtfilt, butter, welch

# === Parameters
sampling_rate = 1000  # Hz
trial_length = 5140   # samples
offset = 140          # samples to skip at trial start

# === Load EEG data
data = pd.read_csv("S1_freq_10Hz.csv")  # Replace with actual filename
channel_name = "Oz"  # Change to another channel if needed
signal = data[channel_name].values

# === Plot PSD for each of 12 trials
plt.figure(figsize=(15, 20))

for trial_idx in range(12):
    start = offset + trial_idx * trial_length
    end = start + trial_length
    trial_data = signal[start:end]

    # Remove DC offset
    trial_data = trial_data - np.mean(trial_data)

    # Apply notch filter at 50 Hz
    b_notch, a_notch = iirnotch(50, Q=30, fs=sampling_rate)
    trial_data = filtfilt(b_notch, a_notch, trial_data)

    # Apply bandpass filter (6–20 Hz typical for SSVEP)
    b_band, a_band = butter(4, [6, 20], btype='bandpass', fs=sampling_rate)
    trial_data = filtfilt(b_band, a_band, trial_data)

    # Compute PSD using Welch’s method
    freqs, psd = welch(trial_data, fs=sampling_rate, nperseg=1024)

    # Plot
    plt.subplot(4, 3, trial_idx + 1)
    plt.plot(freqs, psd, linewidth=1.5)
    plt.title(f"PSD - Trial {trial_idx + 1}")
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Power")
    plt.xlim(0, 30)  # Focus on relevant SSVEP band
    plt.grid(True)

plt.tight_layout()
plt.show()

# Merging Files

In [ ]:
import pandas as pd
import numpy as np

# === Frequencies and file paths
frequencies = [7, 8, 10, 13]
dataframes = []

for freq in frequencies:
    # Load the EEG CSV file
    df = pd.read_csv(f"S1_freq_{hz}Hz.csv")
    dataframes.append(df)

# === Combine all data vertically
merged_data = pd.concat(dataframes, ignore_index=True)

# === Save to CSV
merged_data.to_csv("S1_merged_data.csv", index=False)

print("✅ Done: Saved S1_merged_data.csv and S1_merged_labels.csv")


# Extract CSV of FFTs

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import iirnotch, filtfilt, butter

# === Settings
sampling_rate = 1000
trial_length = 5140
num_trials = 48
frequencies_to_extract = [7, 8, 10, 13, 14, 16, 20, 26]  # Hz

# === Load the merged EEG file (all 8 channels)
data = pd.read_csv("S1_merged_data.csv")

# === Assume you’re using only the Oz channel for FFT
channel_data = data["Oz"].values  # Change channel if needed

# === Pre-allocate output array
fft_features = []

# === Preprocessing filters
# Notch filter (50 Hz)
b_notch, a_notch = iirnotch(50, Q=30, fs=sampling_rate)

# Bandpass filter (covering all relevant SSVEP freqs)
b_band, a_band = butter(4, [6, 30], btype='bandpass', fs=sampling_rate)

# === Loop through each trial
for i in range(num_trials):
    start = i * trial_length + 140
    end = start + trial_length
    trial = channel_data[start:end]

    # Detrend
    trial = trial - np.mean(trial)

    # Filtering
    trial = filtfilt(b_notch, a_notch, trial)
    trial = filtfilt(b_band, a_band, trial)

    # FFT
    n = len(trial)
    fft_vals = np.fft.rfft(trial)
    fft_freqs = np.fft.rfftfreq(n, d=1/sampling_rate)

    # Extract magnitude at target frequencies
    trial_features = []
    for f in frequencies_to_extract:
        # Find closest frequency bin
        idx = (np.abs(fft_freqs - f)).argmin()
        trial_features.append(np.abs(fft_vals[idx]))

    fft_features.append(trial_features)

# === Save output
fft_df = pd.DataFrame(fft_features, columns=[f"{f}Hz" for f in frequencies_to_extract])
fft_df.to_csv("S1_fft_features.csv", index=False)

print("✅ Done: Saved 48-trial FFT feature matrix as S1_fft_features.csv")

# Classification with Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Load the features and target labels
features = pd.read_csv("test.csv")

# Separate the features (FFT values) and the target labels
X = features.drop(columns=['label'])  # FFT values
y = features['label']  # Target labels

# Scale the features (only the FFT values, not the target labels)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(scaled_features, y, test_size=0.2, random_state=42)

# Train the RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predicting on the test set
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")